In [7]:
import psycopg2
import arcpy
import arcgis
import pandas as pd
import numpy as np
import datetime
import requests
import matplotlib.pyplot as plt
import pyodbc
import os
import warnings
import subprocess
import sys
import psycopg2

GET DATA FROM POSTGIS

In [71]:
# Set up the PostgreSQL connection
connection = psycopg2.connect(host = '35.223.186.20',
                             database = 'postgres',
                             user = 'postgres',
                             password = '139571wang')

cur = connection.cursor()
cur.execute("SELECT * FROM temperature_df")
rows = cur.fetchall()

#transform to pd dataframe and aggregate data
rows = pd.DataFrame(rows)
rows.columns = ['id','date','name','max_tmpf','min_tmpf','x','y','month']
rows = rows[rows['month']==3].reset_index(drop=True)

rows_agg = rows.groupby('id')[['max_tmpf','min_tmpf']].mean().reset_index()

rows = pd.merge(left = rows_agg, right=rows[['id','name','x','y']].drop_duplicates(), how='left',left_on=['id'], right_on=['id'])

In [80]:
arcpy.env.workspace = r"C:\Users\Maochuan\OneDrive\文档\ArcGIS\Projects\arc2_lab3\arc2_lab3.gdb"

sr = arcpy.SpatialReference(4326) # WGS 1984

# create a new feature class
arcpy.CreateFeatureclass_management(arcpy.env.workspace, "temperature", "POINT",spatial_reference=sr)

# add fields to the feature class
arcpy.AddField_management(r"C:\Users\Maochuan\OneDrive\桌面\lab3\temperature.shp", "station_id", "Text")
arcpy.AddField_management(r"C:\Users\Maochuan\OneDrive\桌面\lab3\temperature.shp", "name", "Text")
arcpy.AddField_management(r"C:\Users\Maochuan\OneDrive\桌面\lab3\temperature.shp", "max_tmpf", "Double")
arcpy.AddField_management(r"C:\Users\Maochuan\OneDrive\桌面\lab3\temperature.shpp", "min_tmpf", 'Double')

# insert data into the feature class
cursor = arcpy.da.InsertCursor(r"C:\Users\Maochuan\OneDrive\桌面\lab3\temperature.shp", ["SHAPE@", "station_id", "name","max_tmpf",'min_tmpf'])
for index,row in rows.iterrows():
    point = arcpy.Point(row[4], row[5])
    cursor.insertRow([point, row[0], row[3],row[1], row[2]])
del cursor


cur.close()
connection.close()

IDW INTERPORLATION

In [17]:
arcpy.ga.IDW("temperature", "max_tmpf", r"C:\test\temperature_idw.lyr", r"C:\test\temperature_idw.tif", 0.0218500823976, 2, "NBRTYPE=Standard S_MAJOR=2.32298811026612 S_MINOR=2.32298811026612 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", None)

<Result 'C:\\test\\temperature_idw.lyr'>

KRIGING

In [18]:
arcpy.ga.EmpiricalBayesianKriging("temperature", "max_tmpf", r"C:\test\kriging_geo.lyr", r"C:\test\kriging.tif", 0.0218500823976, "NONE", 100, 1, 100, "NBRTYPE=StandardCircular RADIUS=2.32298811026612 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "PREDICTION", 0.5, "EXCEED", None, "POWER")

<Result 'C:\\test\\kriging_geo.lyr'>

RADIAL BASIS FUNCTIONS

In [84]:
arcpy.ga.RadialBasisFunctions("temperature", "max_tmpf", r"C:\test\RBF_geo.lyr", r"C:\test\RBF.tif", 0.0218500823976, "NBRTYPE=Standard S_MAJOR=2.32298811026612 S_MINOR=2.32298811026612 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "COMPLETELY_REGULARIZED_SPLINE", None)

<Result 'temperature_RBF_geo.lyr'>

ASSESMENT (CROSS VALIDATION)

In [20]:
arcpy.ga.CrossValidation(r"C:\test\temperature_idw.lyr", r"C:\test\IDW_CV.shp")
arcpy.ga.CrossValidation(r"C:\test\kriging_geo.lyr", r"C:\test\Kriging_CV.shp")
arcpy.ga.CrossValidation(r"C:\test\RBF_geo.lyr", r"C:\test\RBF_CV.shp")

<Result 'C:\\test\\RBF_CV.shp'>

CALCULATE RMSE AND COMPARE

In [13]:
# Convert attribute table to a NumPy array and pd DataFrame
idw_cv_array = arcpy.da.FeatureClassToNumPyArray(r"C:\test\IDW_CV.shp", ["FID", "Error"])
idw_cv_pd = pd.DataFrame(idw_cv_array, columns=["FID", "Error"])

Kriging_cv_array = arcpy.da.FeatureClassToNumPyArray(r"C:\test\Kriging_cv.shp", ["FID", "Error"])
Kriging_cv_pd = pd.DataFrame(Kriging_cv_array, columns=["FID", "Error"])

RBF_cv_array = arcpy.da.FeatureClassToNumPyArray(r"C:\test\RBF_cv.shp", ["FID", "Error"])
RBF_cv_pd = pd.DataFrame(RBF_cv_array, columns=["FID", "Error"])

# Calculate squared error
idw_cv_pd['squared_error'] = idw_cv_pd['Error'] ** 2
Kriging_cv_pd['squared_error'] = Kriging_cv_pd['Error'] ** 2
RBF_cv_pd['squared_error'] = RBF_cv_pd['Error'] ** 2

# Calculate RMSE
idw_rmse = numpy.sqrt(idw_cv_pd['squared_error'].mean())
Kriging_rmse = numpy.sqrt(Kriging_cv_pd['squared_error'].mean())
RBF_rmse = numpy.sqrt(RBF_cv_pd['squared_error'].mean())

#Print and compare results
print(f'The RMSE for IDW is: {idw_rmse}')
print(f'The RMSE for Kriging is: {Kriging_rmse}')
print(f'The RMSE for RBF is: {RBF_rmse}')

The RMSE for IDW is: 1.5042421137517141
The RMSE for Kriging is: 1.4072889938385105
The RMSE for RBF is: 1.4526274613313865


In [ ]:
# Set up the PostgreSQL connection
connection = psycopg2.connect(host = '35.223.186.20',
                             database = 'postgres',
                             user = 'postgres',
                             password = '139571wang')

# Set the name of the new table in the PostgreSQL database
output_table = "temperature_assesment_table"

# Use psycopg2 to create the table in the PostgreSQL database
cursor = connection.cursor()
cursor.execute(f"CREATE TABLE {output_table} (method text, rmse double precision)")

# Use psycopg2 to copy the data from the Geodatabase table to the PostgreSQL table
cursor.execute(f"INSERT INTO {output_table} (method, rmse) VALUES (%s, %s)", ('IDW',idw_rmse))
cursor.execute(f"INSERT INTO {output_table} (method, rmse) VALUES (%s, %s)", ('Kriging',Kriging_rmse))
cursor.execute(f"INSERT INTO {output_table} (method, rmse) VALUES (%s, %s)", ('RBF',RBF_rmse))


# Commit the changes to the PostgreSQL database
connection.commit()

# Close the PostgreSQL connection and cursor
cursor.close()
connection.close()

In [34]:
import arcpy
from arcpy.sa import *

# Define input temperature data shapefile
input_shapefile = r"C:\test\temperature.shp"

# Define the max temperature field
max_temperature_field = "max_tmpf"

# Define Kriging interpolation raster
kriging_output = r"C:\test\kriging.tif"

# Define output difference point layer
output_shapefile = r"C:\test\tmp_krig_difference_point_layer.shp"

# Copy the original point data to the new point layer
arcpy.CopyFeatures_management(input_shapefile, output_shapefile)

# Extract Kriging values at point locations
kriging_diff_temp = r"C:\test\temp_krig_diff.shp"
ExtractValuesToPoints(output_shapefile, kriging_output, kriging_diff_temp)

# Add a new field called "difference" to store the difference between max_tmpf and RASTERVALU
arcpy.AddField_management(kriging_diff_temp, "difference", "DOUBLE")

# Update the "difference" field with the value of max_tmpf - RASTERVALU for each row
with arcpy.da.UpdateCursor(kriging_diff_temp, ["max_tmpf", "RASTERVALU", "difference"]) as cursor:
    for row in cursor:
        row[2] = row[0] - row[1]
        cursor.updateRow(row)


# Upload kriging to postgis database

In [21]:
arcpy.ddd.Int(r"C:\test\kriging.tif", r"C:\test\kriging_temperature_int.tif")

In [ ]:
arcpy.conversion.RasterToPolygon(r"C:\test\kriging_temperature_int.tif", r"C:\test\kriging_polygon.shp", "SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)

In [ ]:
arcpy.analysis.Clip("kriging_polygon", r"BOUNDARIES OF MINNESOTA\STATE", r"C:\test\minnesota_kriging.shp", None)

In [ ]:
arcpy.management.Project("minnesota_kriging", r"C:\test\Kriging_Projected.shp", 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

In [22]:
import subprocess
import os

# Set the necessary parameters
shp2pgsql_path = r"C:\Program Files\PostgreSQL\15\bin\shp2pgsql.exe"
psql_path = r"C:\Program Files\PostgreSQL\15\bin\psql.exe"
shapefile_path = r"C:\test\Kriging_Projected.shp"
srid = 4326
schema_table = "public.kriging_temperature"
host = "35.223.186.20"
port = 5432
user = "postgres"
db_name = "postgres"
password = "139571wang"

# Build the shp2pgsql command
shp2pgsql_command = f"\"{shp2pgsql_path}\" -s {srid} {shapefile_path} {schema_table}"

# Build the psql command
psql_command = f"\"{psql_path}\" -h {host} -p {port} -U {user} -d {db_name} -w"

# Combine the commands using a pipe
full_command = f"{shp2pgsql_command} | {psql_command}"

# Set up the environment with the password
env = os.environ.copy()
env["PGPASSWORD"] = password

# Run the command using the subprocess module and capture stderr
process = subprocess.Popen(full_command, shell=True, env=env, stderr=subprocess.PIPE)

# Wait for the process to finish and capture the return code and stderr
return_code = process.wait()
stderr_output = process.stderr.read().decode()

# Print the return code and error message
print(f"Return code: {return_code}")
print(f"Error message: {stderr_output}")


Return code: 0
Error message: could not print result table: Invalid argument
Field id is an FTDouble with width 10 and precision 0
Field gridcode is an FTDouble with width 10 and precision 0
Shapefile type: Polygon
Postgis type: MULTIPOLYGON[2]



In [ ]:
import psycopg2
from flask import Flask, jsonify
from psycopg2 import sql
from psycopg2.extras import RealDictCursor

app = Flask(__name__)

@app.route('/temperature_kriging', methods=['GET'])
def get_polygon():
    conn = psycopg2.connect(host='35.223.186.20',
                            database='postgres',
                            user='postgres',
                            password='139571wang')

    cur = conn.cursor(cursor_factory=RealDictCursor)
    cur.execute(sql.SQL("SELECT gridcode, ST_AsGeoJSON(geom)::json AS geometry FROM kriging_temperature"))
    result = cur.fetchall()
    cur.close()
    conn.close()
    return jsonify(result)

if __name__ == '__main__':
    app.run(debug=True)


In [36]:
import requests
import json
import os

# Replace this URL with the address of your Flask application
url = "http://34.27.164.176:5000/temperature_kriging"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    geojson_data = response.json()

    # Save the GeoJSON to the specified folder and name it "lab3_test.geojson"
    output_folder = r"C:\Users\Maochuan\OneDrive\桌面\lab3"
    output_file_path = os.path.join(output_folder, "temperature_kriging.geojson")
    print(f"Output file path: {output_file_path}")

    with open(output_file_path, "w") as f:
        json.dump(geojson_data, f)
    print(f"GeoJSON file saved as {output_file_path}")
else:
    print("Error fetching GeoJSON data")


Output file path: C:\Users\Maochuan\OneDrive\桌面\lab3\temperature_kriging.geojson
GeoJSON file saved as C:\Users\Maochuan\OneDrive\桌面\lab3\temperature_kriging.geojson


In [39]:
import json

# Read input JSON data from file
with open(r'C:\Users\Maochuan\OneDrive\桌面\lab3\temperature_kriging.geojson', 'r') as input_file:
    input_data = json.load(input_file)

# Process input data and create output data
output_data = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "type": "MultiPolygon",
                "coordinates": polygon_data["geometry"]["coordinates"]
            },
            "properties": {
                "temperature": polygon_data["gridcode"]
            }
        }
        for polygon_data in input_data
    ]
}

# Write output JSON data to file
with open(r'C:\Users\Maochuan\OneDrive\桌面\lab3\temperature_kriging_s.json', 'w') as output_file:
    json.dump(output_data, output_file, indent=2)


In [40]:
arcpy.conversion.JSONToFeatures(r"C:\Users\Maochuan\OneDrive\桌面\lab3\temperature_kriging_s.json", r"C:\test\temperature_kriging_final.shp", "POLYGON")

<Result 'C:\\test\\temperature_kriging_final.shp'>

In [ ]:
import os
import zipfile
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

# Function to zip shapefile
def zip_shapefile(shapefile_path):
    with zipfile.ZipFile(shapefile_path + '.zip', 'w', zipfile.ZIP_DEFLATED) as zf:
        for file_name in os.listdir(os.path.dirname(shapefile_path)):
            if file_name.startswith(os.path.basename(shapefile_path).split('.')[0]) and not file_name.endswith('.zip'):
                zf.write(os.path.join(os.path.dirname(shapefile_path), file_name), arcname=file_name)

# Connect to your ArcGIS Online organization
gis = GIS("home")

# Set the input shapefile path
input_shapefile = r"C:\test\temperature_kriging_final.shp"

# Zip the shapefile
zip_shapefile(input_shapefile)

# Upload the zipped shapefile as a new item
item = gis.content.add({'type': 'Shapefile'}, input_shapefile + '.zip')

# Publish the new item as a feature layer
feature_layer_item = item.publish()

# Get the feature layer URL and print it
feature_layer_url = feature_layer_item.url
print(feature_layer_url)


# upload kriging accuracy assessment layer to postgis and upload to arconline

In [35]:
import subprocess
import os

# Set the necessary parameters
shp2pgsql_path = r"C:\Program Files\PostgreSQL\15\bin\shp2pgsql.exe"
psql_path = r"C:\Program Files\PostgreSQL\15\bin\psql.exe"
shapefile_path = r"C:\test\tmp_krig_difference.shp"
srid = 4326
schema_table = "public.kriging_temperature_difference"
host = "35.223.186.20"
port = 5432
user = "postgres"
db_name = "postgres"
password = "139571wang"

# Build the shp2pgsql command
shp2pgsql_command = f"\"{shp2pgsql_path}\" -s {srid} {shapefile_path} {schema_table}"

# Build the psql command
psql_command = f"\"{psql_path}\" -h {host} -p {port} -U {user} -d {db_name} -w"

# Combine the commands using a pipe
full_command = f"{shp2pgsql_command} | {psql_command}"

# Set up the environment with the password
env = os.environ.copy()
env["PGPASSWORD"] = password

# Run the command using the subprocess module and capture stderr
process = subprocess.Popen(full_command, shell=True, env=env, stderr=subprocess.PIPE)

# Wait for the process to finish and capture the return code and stderr
return_code = process.wait()
stderr_output = process.stderr.read().decode()

# Print the return code and error message
print(f"Return code: {return_code}")
print(f"Error message: {stderr_output}")


Return code: 0
Error message: could not print result table: Invalid argument
Field max_tmpf is an FTDouble with width 19 and precision 11
Field min_tmpf is an FTDouble with width 19 and precision 11
Field rastervalu is an FTDouble with width 13 and precision 11
Field difference is an FTDouble with width 19 and precision 11
Shapefile type: Point
Postgis type: POINT[2]



In [ ]:
import psycopg2
from flask import Flask, jsonify
from psycopg2 import sql
from psycopg2.extras import RealDictCursor

app = Flask(__name__)

@app.route('/temperature_kriging_difference', methods=['GET'])
def get_polygon():
    conn = psycopg2.connect(host='35.223.186.20',
                            database='postgres',
                            user='postgres',
                            password='139571wang')

    cur = conn.cursor(cursor_factory=RealDictCursor)
    cur.execute(sql.SQL("SELECT difference, ST_AsGeoJSON(geom)::json AS geometry FROM kriging_temperature_difference"))
    result = cur.fetchall()
    cur.close()
    conn.close()
    return jsonify(result)

if __name__ == '__main__':
    app.run(debug=True)


In [31]:
import requests
import json
import os

# Replace this URL with the address of your Flask application
url = "http://34.27.164.176:5000/temperature_kriging_difference"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    geojson_data = response.json()

    # Save the GeoJSON to the specified folder and name it "lab3_test.geojson"
    output_folder = r"C:\Users\Maochuan\OneDrive\桌面\lab3"
    output_file_path = os.path.join(output_folder, "temperature_kriging_difference.geojson")
    print(f"Output file path: {output_file_path}")

    with open(output_file_path, "w") as f:
        json.dump(geojson_data, f)
    print(f"GeoJSON file saved as {output_file_path}")
else:
    print("Error fetching GeoJSON data")


Output file path: C:\Users\Maochuan\OneDrive\桌面\lab3\temperature_kriging_difference.geojson
GeoJSON file saved as C:\Users\Maochuan\OneDrive\桌面\lab3\temperature_kriging_difference.geojson


In [33]:
import json

# Read input JSON data from file
with open(r'C:\Users\Maochuan\OneDrive\桌面\lab3\temperature_kriging_difference.geojson', 'r') as input_file:
    input_data = json.load(input_file)

# Process input data and create output data
output_data = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": point_data["geometry"]["coordinates"]
            },
            "properties": {
                "Error": point_data["error"]
            }
        }
        for point_data in input_data
    ]
}

# Write output JSON data to file
with open(r'C:\Users\Maochuan\OneDrive\桌面\lab3\temperature_kriging_difference_final.json', 'w') as output_file:
    json.dump(output_data, output_file, indent=2)


In [ ]:
arcpy.conversion.JSONToFeatures(r"C:\Users\Maochuan\OneDrive\桌面\lab3\temperature_kriging_difference_final.json", r"C:\test\temperature_kriging_difference_final.shp", "POLYGON")

In [ ]:
import os
import zipfile
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

# Function to zip shapefile
def zip_shapefile(shapefile_path):
    with zipfile.ZipFile(shapefile_path + '.zip', 'w', zipfile.ZIP_DEFLATED) as zf:
        for file_name in os.listdir(os.path.dirname(shapefile_path)):
            if file_name.startswith(os.path.basename(shapefile_path).split('.')[0]) and not file_name.endswith('.zip'):
                zf.write(os.path.join(os.path.dirname(shapefile_path), file_name), arcname=file_name)

# Connect to your ArcGIS Online organization
gis = GIS("home")

# Set the input shapefile path
input_shapefile = r"C:\test\temperature_kriging_difference_final.shp"

# Zip the shapefile
zip_shapefile(input_shapefile)

# Upload the zipped shapefile as a new item
item = gis.content.add({'type': 'Shapefile'}, input_shapefile + '.zip')

# Publish the new item as a feature layer
feature_layer_item = item.publish()

# Get the feature layer URL and print it
feature_layer_url = feature_layer_item.url
print(feature_layer_url)
